In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [44]:
B = 256
C = 4
D = 17
H = 64
W = 64
class BiLSTM2D(nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn_v = nn.GRU(C, D, 
                            num_layers=1, 
                            batch_first=True, 
                            bias=True,
                            bidirectional=True)
        self.rnn_h = nn.GRU(C, D, 
                            num_layers=1, 
                            batch_first=True, 
                            bias=True,
                            bidirectional=True)
    
    def forward(self, x):
        v, _ = self.rnn_v(x.permute(0, 2, 1, 3).reshape(-1, H, C))
        v = v.reshape(B, W, H, -1).permute(0, 2, 1, 3)
        h, _ = self.rnn_h(x.reshape(-1, W, C))
        h = h.reshape(B, H, W, -1)
        x = torch.cat([v, h], dim=-1)
        print(x.shape)
        x = x.reshape(*x.shape[:-1], C, D)
        print(x.shape)
        # x = x.mean(dim=-1, keepdim=True)
        # print(x.shape)
        return x

net = BiLSTM2D()

In [45]:
batched_obs = torch.randn([256, 64, 64, 4])
batched_obs.shape

torch.Size([256, 64, 64, 4])

In [46]:
weight = net(batched_obs)
weight.shape

torch.Size([256, 64, 64, 68])
torch.Size([256, 64, 64, 4, 17])


torch.Size([256, 64, 64, 4, 17])

In [42]:
def bmm_input(b_weight, b_input):
    bmm = torch.einsum('bhwcf, bhwc -> bf', b_weight, b_input)
    return bmm

In [43]:
hidden = bmm_input(weight, batched_obs)

In [7]:
hidden.shape

torch.Size([256, 17])

In [8]:
hidden

tensor([[ 560.6110,  284.6977,  679.4728,  ...,  146.1381,   12.0559,
         -264.9316],
        [ 566.2153,  252.6550,  710.1396,  ...,  147.5023,   77.9313,
         -299.2885],
        [ 583.9216,  228.4675,  647.5270,  ...,  136.6174,   93.6364,
         -342.7438],
        ...,
        [ 577.6134,  269.4700,  667.4359,  ...,  140.9128,   48.1660,
         -271.1786],
        [ 540.0983,  254.1015,  709.4675,  ...,  215.7526,   85.6181,
         -318.4952],
        [ 537.0328,  281.9046,  760.7954,  ...,  127.5297,   52.0008,
         -291.7527]], grad_fn=<ViewBackward0>)

In [9]:
import numpy as np

In [10]:
normalized = hidden / np.sqrt(W*H*D)

In [11]:
normalized

tensor([[ 2.1245,  1.0789,  2.5749,  ...,  0.5538,  0.0457, -1.0040],
        [ 2.1457,  0.9575,  2.6912,  ...,  0.5590,  0.2953, -1.1342],
        [ 2.2128,  0.8658,  2.4539,  ...,  0.5177,  0.3548, -1.2989],
        ...,
        [ 2.1889,  1.0212,  2.5293,  ...,  0.5340,  0.1825, -1.0277],
        [ 2.0468,  0.9629,  2.6886,  ...,  0.8176,  0.3245, -1.2070],
        [ 2.0352,  1.0683,  2.8831,  ...,  0.4833,  0.1971, -1.1056]],
       grad_fn=<DivBackward0>)

In [33]:
B = 256
C = 4
D = 17
H = 64
W = 64
class BiLSTM2D(nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn_c = nn.GRU(1, D, 
                            num_layers=1, 
                            batch_first=True, 
                            bias=True,
                            bidirectional=True)
        self.rnn_v = nn.GRU(D, D, 
                            num_layers=1, 
                            batch_first=True, 
                            bias=True,
                            bidirectional=True)
        self.rnn_h = nn.GRU(D, D, 
                            num_layers=1, 
                            batch_first=True, 
                            bias=True,
                            bidirectional=True)
    
    def forward(self, x, H, W, C):
        x, _ = self.rnn_c(x.permute(0, 3, 2, 1).reshape(-1, C, 1))
        print(x.shape)
        x = x.reshape(-1, H, W, 1)
        print(x.shape)
        v, _ = self.rnn_v(x.permute(0, 2, 1, 3).reshape(-1, H, D))
        v = v.reshape(B, W, H, -1).permute(0, 2, 1, 3)
        h, _ = self.rnn_h(x.reshape(-1, W, D))
        h = h.reshape(B, H, W, -1)
        x = torch.cat([v, h], dim=-1)
        print(x.shape)
        x = x.reshape(*x.shape[:-1], 1, -1)
        print(x.shape)
        # x = x.mean(dim=-1, keepdim=True)
        # print(x.shape)
        return x

net = BiLSTM2D()

In [34]:
batched_obs = torch.randn([256, 64, 64, 4])
batched_obs.shape

torch.Size([256, 64, 64, 4])

In [35]:
weight = net(batched_obs, 64, 64, 4)
weight.shape

torch.Size([1048576, 4, 34])
torch.Size([34816, 64, 64, 1])
torch.Size([256, 64, 64, 544])
torch.Size([256, 64, 64, 1, 544])


torch.Size([256, 64, 64, 1, 544])

In [32]:
570425344 / 256 / 64 / 64

544.0

In [38]:
17 * 17 * 2

578